In [2]:
%%writefile app/model.py
import torch
import torchvision

def create_vit(num_classes: int=6,
               seed: int=42):
    weights = torchvision.models.ViT_B_16_Weights.DEFAULT
    
    transforms = weights.transforms()
    
    model = torchvision.models.vit_b_16(weights=weights)
    
    for param in model.parameters():
        param.requires_grad = False
        
    torch.manual_seed(seed)
    model.heads = torch.nn.Sequential(torch.nn.LayerNorm(normalized_shape=768),
                                      torch.nn.Linear(in_features=768, out_features=num_classes))
    return model, transforms

Overwriting app/model.py


In [20]:
import shutil
from pathlib import Path

# 1. Create an examples directory
app = Path("app/")
scene_recog_app_path = app / "examples"
scene_recog_app_path.mkdir(parents=True, exist_ok=True)

# 2. Collect three random test dataset image paths
scene_recog_examples = [Path("scenes/seg_test/buildings/20140.jpg"),
                            Path("scenes/seg_test/forest/20225.jpg"),
                            Path("scenes/seg_test/glacier/20087.jpg"), 
                            Path("scenes/seg_test/mountain/20176.jpg"),
                            Path("scenes/seg_test/sea/20247.jpg"),
                            Path("scenes/seg_test/street/20080.jpg")]

# 3. Copy the three random images to the examples directory
for example in scene_recog_examples:
    destination = scene_recog_app_path / example.name
    print(f"[INFO] Copying {example} to {destination}")
    shutil.copy2(src=example, dst=destination)

[INFO] Copying scenes\seg_test\buildings\20140.jpg to app\examples\20140.jpg
[INFO] Copying scenes\seg_test\forest\20225.jpg to app\examples\20225.jpg
[INFO] Copying scenes\seg_test\glacier\20087.jpg to app\examples\20087.jpg
[INFO] Copying scenes\seg_test\mountain\20176.jpg to app\examples\20176.jpg
[INFO] Copying scenes\seg_test\sea\20247.jpg to app\examples\20247.jpg
[INFO] Copying scenes\seg_test\street\20080.jpg to app\examples\20080.jpg


In [4]:
%%writefile app/app.py
import gradio as gr
import os
import torch

from model import create_vit
from timeit import default_timer as timer
from typing import Tuple, Dict

class_names = ["buildings", "forest", "glacier", "mountain", "sea", "street"]

vit_model, vit_transforms = create_vit(num_classes=len(class_names),
                                       seed=42)

vit_model.load_state_dict(
    torch.load(
        f="pretrained_vit_feature_extractor_scene_recognition.pth", 
        map_location=torch.device("cpu")
    )
)

def predict(img):
    start_timer = timer()
    
    img = vit_transforms(img).unsqueeze(0)
    
    vit_model.eval()
    with torch.inference_mode():
        pred_prob = torch.softmax(vit_model(img), dim=1)
        
    pred_labels_and_probs = {class_names[i]: float(pred_prob[0][i]) for i in range(len(class_names))}
    
    pred_time = round(timer() - start_timer, 5)
    
    return pred_labels_and_probs, pred_time

title = "Scene Recognition: Intel Image Classification"
description = "A ViT feature extractor Computer Vision model to classify images of scenes from 1 out of 6 classes."
article = "Access project repository at [GitHub](https://github.com/Ammar2k/intel_image_classification)"

example_list = [["examples/" + example] for example in os.listdir("examples")]

demo = gr.Interface(fn=predict, 
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=6, label="Predictions"), 
                    gr.Number(label="Prediction time(s)")],
                    examples=example_list,
                    title=title,
                    description=description,
                    article=article
                   )

demo.launch()

Writing app/app.py


In [7]:
# import gradio as gr
# import os
# import torch

# from app.model import create_vit
# from timeit import default_timer as timer
# from typing import Tuple, Dict

# from torchinfo import summary

# class_names = ["buildings", "forest", "glacier", "mountain", "sea", "street"]

# vit_model, vit_transforms = create_vit(num_classes=len(class_names),
#                                        seed=42)

# vit_model.load_state_dict(
#     torch.load(
#         f="app/pretrained_vit_feature_extractor_scene_recognition.pth", 
#         map_location=torch.device("cpu")
#     )
# )

# def predict(img):
#     start_timer = timer()
    
#     img = vit_transforms(img).unsqueeze(0)
    
#     vit_model.eval()
#     with torch.inference_mode():
#         pred_prob = torch.softmax(vit_model(img), dim=1)
        
#     pred_labels_and_probs = {class_names[i]: round(pred_prob[0][i].item(), 3) for i in range(len(class_names))}
    
#     pred_time = round(timer() - start_timer, 5)
    
#     return pred_labels_and_probs, pred_time

# title = "Scene Recognition: Intel Image Classification"
# description = "A ViT feature extractor Computer Vision model to classify images of scenes from 1 out of 6 classes."
# article = "Access project repository at [GitHub](https://github.com/Ammar2k/intel_image_classification)"

# example_list = [["app/examples/" + example] for example in os.listdir("app/examples")]

# demo = gr.Interface(fn=predict, 
#                     inputs=gr.Image(type="pil"),
#                     outputs=[gr.Label(num_top_classes=6, label="Predictions"), 
#                     gr.Number(label="Prediction time(s)")],
#                     examples=example_list,
#                     title=title,
#                     description=description,
#                     article=article
#                    )

# demo.launch(debug=False, share=True)

In [10]:
%%writefile app/requirements.txt
torch
torchvision
gradio

Writing app/requirements.txt
